# Does BLOSUM62 satisfy the Triangle Inequality?

In [4]:
import itertools
import math

In [1]:
!wget -c 'https://drive.google.com/uc?export=download&id=1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy' -O data.zip
!unzip -o data.zip 

--2022-12-15 20:04:41--  https://drive.google.com/uc?export=download&id=1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy
Resolving drive.google.com (drive.google.com)... 108.177.111.138, 108.177.111.113, 108.177.111.101, ...
Connecting to drive.google.com (drive.google.com)|108.177.111.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/96qbgsmp6tn5icj68rbq49bunu5nfes8/1671134625000/07504575957156967943/*/1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy?e=download&uuid=8d20e8c7-3bfe-403e-93d3-d50a16d36bb3 [following]
--2022-12-15 20:04:42--  https://doc-0c-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/96qbgsmp6tn5icj68rbq49bunu5nfes8/1671134625000/07504575957156967943/*/1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy?e=download&uuid=8d20e8c7-3bfe-403e-93d3-d50a16d36bb3
Resolving doc-0c-3c-docs.googleusercontent.com (doc-0c-3c-docs.googleusercontent.com)... 108.177.111.132, 

In [3]:
def read_blosum62(path):
    """
    Reads in the ncbi's BLOSUM62.txt file and loads the scoring matrix
    into a dictionary.
    
    :param: path is the full path in the local filesystem at which the .txt file is located
    :return: a dictionary of dictionaries which will hold the cost of various amino acid
    substitutions as defined in BLOSUM62.
    """
    delta = {}
    with open(path, 'r') as f:
        lines = f.readlines()[6:]
        keys = lines[0].split()
        keys[-1] = '-'
        for i, line in enumerate(lines[1:]):
            delta[keys[i]] = {k : int(v) for (k,v) in zip(keys, line.split()[1:])}  
    return delta

blosum = read_blosum62('./BLOSUM62.txt')
alphabet = [k for k in blosum]

In [5]:
tally = {c:0 for c in alphabet}
count = 0

for A, B, C in itertools.combinations(alphabet, 3):
    if not abs(blosum[A][B]) + abs(blosum[B][C]) >= abs(blosum[A][C]):
        # print(f'({A},{B},{C}): {abs(blosum[A][B])} + {abs(blosum[B][C])} < {abs(blosum[A][C])}')
        
        tally[A], tally[B], tally[C] = tally[A] + 1, tally[B] + 1, tally[C] + 1
        count += 1
        

print(f'{count}, {count/2024}%')
print(tally)

174, 0.08596837944664032%
{'A': 17, 'R': 20, 'N': 40, 'D': 26, 'C': 11, 'Q': 33, 'E': 26, 'G': 16, 'H': 24, 'I': 13, 'L': 13, 'K': 20, 'M': 26, 'F': 25, 'P': 12, 'S': 40, 'T': 32, 'W': 25, 'Y': 5, 'V': 24, 'B': 38, 'Z': 26, 'X': 10, '-': 0}


In [11]:
tally = {c:0 for c in alphabet}
count = 0

for A in alphabet:
    for B, C in itertools.combinations(alphabet, 2):
        if not abs(blosum[A][B]) + abs(blosum[B][C]) >= abs(blosum[A][C]):
            # print(f'({A},{B},{C}): {abs(blosum[A][B])} + {abs(blosum[B][C])} < {abs(blosum[A][C])}')
            
            if A == B:
                tally[A], tally[C] = tally[A] + 1, tally[C] + 1
            else:
                tally[A], tally[B], tally[C] = tally[A] + 1, tally[B] + 1, tally[C] + 1
            count += 1


print(f'{count}, {count/6624}%')
print(tally)

681, 0.10280797101449275%
{'A': 150, 'R': 81, 'N': 116, 'D': 57, 'C': 60, 'Q': 116, 'E': 87, 'G': 88, 'H': 100, 'I': 47, 'L': 61, 'K': 69, 'M': 88, 'F': 73, 'P': 68, 'S': 88, 'T': 92, 'W': 77, 'Y': 48, 'V': 85, 'B': 111, 'Z': 87, 'X': 27, '-': 0}


# Affine Gap Penalty Alignment

\begin{equation}
s^\rightarrow[i,j] = \max
\begin{cases}
s^\rightarrow[i,j-1] - \sigma, & \mbox{if j > 1,} \\
s^\searrow[i,j-1] - (\sigma + \rho), & \mbox{if j > 0,}
\end{cases}
\end{equation}

\begin{equation}
s^\searrow[i,j] = \max
\begin{cases}
0, & \mbox{if i = 0 and j = 0,} \\
s^\rightarrow[i,j], & \mbox{if j > 0,} \\
s^\downarrow[i,j], & \mbox{if i > 0,} \\
s^\searrow[i-1,j-1] - \delta(v_i,w_i), & \mbox{if i > 0 and j > 0,}
\end{cases}
\end{equation}

\begin{equation}
s^\downarrow[i,j] = \max
\begin{cases}
s^\downarrow[i-1,j] - \sigma, & \mbox{if i > 1,} \\
s^\searrow[i-1,j] - (\sigma + \rho), & \mbox{if i > 0.}
\end{cases}
\end{equation}

Great project! For real data, look at PFAM and HOMSTRAD.

In [ ]:
ORIGIN = (0, 0, 0)
UP = (-1, 0, 0)
LEFT = (0, -1, 0)
TOPLEFT = (-1, -1, 0)
DELOPEN = (0, -1, 1)
DELCLOSE = (0, 0, -1)
INSOPEN = (-1, 0, -1)
INSCLOSE = (0, 0, 1)


def traceback_global(v, w, pointers):
    i, j = len(v), len(w)
    new_v, new_w = [], []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    return ''.join(new_v[::-1]) + '\n' + ''.join(new_w[::-1])


def global_align(v, w, delta, sigma, rho):
    """
    Returns the score of the maximum scoring alignment of the strings v and w, as well as the actual alignment as 
    computed by traceback_global. 
    
    :param: v
    :param: w
    :param: delta
    :param: sigma
    :param: rho
    """

    M_h = M_d = M_v = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    P_h = P_d = P_v = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)]

    for i in range(len(v)+1):
        for j in range(len(w)+1):
            # Horizontal Table
            extend_gap = M_h[i][j-1] - sigma if j > 1 else float('-inf')
            open_gap = M_d[i][j-1] - (sigma + rho) if j > 0 else float('-inf')
            M_h[i][j] = max(extend_gap, open_gap)

            # Diagonal Table
            orig = 0 if i == 0 and j == 0 else float('-inf')
            close_del = M_h[i][j] if j > 0 else float('-inf')
            close_ins = M_v[i][j] if i > 0 else float('-inf')
            sub = M_d[i-1][j-1] + delta[v[i-1]][w[j-1]] if i > 0 and j > 0 else float('-inf')
            M_d[i][j] = max(orig, close_del, close_ins, sub)

            # Vertical Table
            extend_gap = M_v[i-1][j] - sigma if i > 1 else float('-inf')
            open_gap = M_d[i-1][j] - (sigma + rho) if i > 0 else float('-inf')
            M_v[i][j] = max(extend_gap, open_gap)

    score = M_d[-1][-1]
    # alignment = traceback_global(v, w, pointers)
    return score #, alignment

In [ ]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
    
global_align("TAGATA", "GTAGGCTTAAGGTTA", delta, 1, 1)

2